# 🤖 OpenAI Agents SDK 기반 RAG 챗봇 실습

이번 실습에서는 **OpenAI Agents SDK**를 활용하여 RAG(Retrieval-Augmented Generation) 챗봇을 구축해보겠습니다.

> 📢 **OpenAI Agents SDK란?**
> 
> OpenAI Agents SDK는 **멀티 에이전트 워크플로우**를 구축하기 위한 프레임워크입니다.
> - **Agent**: 지시사항(instructions)과 도구(tools)를 갖춘 LLM 기반 에이전트
> - **Handoff**: 에이전트 간 대화 제어권 이전
> - **Tools**: 웹 검색, 파일 검색, 커스텀 함수 등 다양한 도구
> - **Guardrails**: 입력/출력 검증을 위한 안전장치
> - **Tracing**: 실행 흐름 추적 및 디버깅


---
## 1. 환경 설정

먼저 필요한 패키지들을 설치하고 환경을 구성합니다.

### 주요 패키지:
- **openai-agents**: OpenAI Agents SDK
- **openai**: OpenAI Python 클라이언트 (Vector Store 관리용)


In [ ]:
# 필요한 패키지 설치
# %pip install -qU openai-agents openai python-dotenv

In [48]:
# 환경 변수 설정
import os
from dotenv import load_dotenv

load_dotenv()

# OpenAI API 키 설정 (환경 변수에 없으면 직접 입력)
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

print("✅ 환경 설정 완료!")


✅ 환경 설정 완료!


---
## 2. Vector Store 생성

OpenAI의 Vector Store는 **파일 검색을 위한 벡터 인덱스 저장소**입니다.

### Vector Store 특징:
- OpenAI 서버에서 관리되는 **호스팅 벡터 스토어**
- 자동 청킹 및 임베딩 처리
- FileSearchTool과 연동하여 RAG 구현


In [49]:
from openai import OpenAI

# OpenAI 클라이언트 초기화
client = OpenAI()

# Vector Store 생성
vector_store = client.vector_stores.create(
    name="deepseek-ocr-docs"
)

print(f"📦 Vector Store 생성 완료!")
print(f"   - ID: {vector_store.id}")
print(f"   - Name: {vector_store.name}")
print(f"   - Status: {vector_store.status}")


📦 Vector Store 생성 완료!
   - ID: vs_696257182bc48191a31b7850b0ae2c2a
   - Name: deepseek-ocr-docs
   - Status: completed


---
## 3. PDF 파일 업로드 및 인덱싱

PDF 문서를 Vector Store에 업로드하고 인덱싱합니다.

### 지원되는 파일 형식:
- PDF, TXT, DOCX, PPTX
- MD, HTML, JSON
- 그 외 다양한 텍스트 기반 파일


In [50]:
# PDF 파일 경로 설정
pdf_path = "..\docs\DeepSeek_OCR_paper.pdf"

# 파일 업로드
with open(pdf_path, "rb") as f:
    file_upload = client.files.create(
        file=f,
        purpose="assistants"
    )

print(f"📄 파일 업로드 완료!")
print(f"   - File ID: {file_upload.id}")
print(f"   - Filename: {file_upload.filename}")
print(f"   - Size: {file_upload.bytes:,} bytes")


📄 파일 업로드 완료!
   - File ID: file-BEUc9vAdrmPsMZmXSc9pjT
   - Filename: DeepSeek_OCR_paper.pdf
   - Size: 7,591,202 bytes


In [51]:
# Vector Store에 파일 인덱싱 (완료될 때까지 대기)
indexed_file = client.vector_stores.files.create_and_poll(
    vector_store_id=vector_store.id,
    file_id=file_upload.id
)

print(f"🔍 파일 인덱싱 완료!")
print(f"   - Status: {indexed_file.status}")
print(f"   - Vector Store ID: {indexed_file.vector_store_id}")


🔍 파일 인덱싱 완료!
   - Status: completed
   - Vector Store ID: vs_696257182bc48191a31b7850b0ae2c2a


In [52]:
# Vector Store 상태 확인
vs_status = client.vector_stores.retrieve(vector_store.id)

print(f"📊 Vector Store 상태:")
print(f"   - 총 파일 수: {vs_status.file_counts.total}")
print(f"   - 완료된 파일: {vs_status.file_counts.completed}")
print(f"   - 처리 중: {vs_status.file_counts.in_progress}")


📊 Vector Store 상태:
   - 총 파일 수: 1
   - 완료된 파일: 1
   - 처리 중: 0


---
## 4. FileSearchTool 에이전트 생성

**FileSearchTool**은 LLM이 Vector Store를 검색할 수 있게 해주는 **호스팅 도구**입니다.

### FileSearchTool 주요 파라미터:
- `vector_store_ids`: 검색할 Vector Store ID 목록
- `max_num_results`: 최대 검색 결과 수
- `include_search_results`: 검색 결과를 출력에 포함할지 여부

```python
FileSearchTool(
    vector_store_ids=["vs_xxx"],  # Vector Store ID
    max_num_results=5,             # 최대 5개 결과
    include_search_results=True    # 검색 결과 포함
)
```


In [ ]:
from agents import Agent, FileSearchTool, Runner, trace

rag_agent = Agent(
    name="RAG Expert",
    instructions = "You are a helpful assistant that answers questions based on the documents in the vector store."
    tools = [FileSearchTool(vector_store_ids=[vector_store.id], max_num_results=5, include_search_results=True)]
)


In [53]:
from agents import Agent, FileSearchTool, Runner, trace

# RAG 에이전트 생성
rag_agent = Agent(
    name="RAG Expert",
    instructions="""You are a helpful assistant that answers questions based on the documents in the vector store.
    
    Important guidelines:
    - Always search the vector store before answering
    - Only answer based on the information found in the documents
    - If you can't find relevant information, say so honestly
    - Provide concise but comprehensive answers
    - Answer in Korean
    """,
    tools=[
        FileSearchTool(
            vector_store_ids=[vector_store.id],
            max_num_results=5,
            include_search_results=True
        )
    ]
)

print(f"🤖 RAG 에이전트 생성 완료!")
print(f"   - Name: {rag_agent.name}")
print(f"   - Tools: {[tool.name for tool in rag_agent.tools]}")


🤖 RAG 에이전트 생성 완료!
   - Name: RAG Expert
   - Tools: ['file_search']


In [54]:
async def ask_rag_agent(question: str):
    """RAG 에이전트에게 질문하기"""
    with trace("RAG Query"):
        result = await Runner.run(rag_agent, question)
        return result

# 첫 번째 질문
question1 = "DeepSeek OCR이 뭐야?"

print(f"💬 질문: {question1}\n")
print("=" * 50)

result1 = await ask_rag_agent(question1)

print(f"\n🤖 답변:\n{result1.final_output}")

💬 질문: DeepSeek OCR이 뭐야?


🤖 답변:
DeepSeek OCR는 LLM(대형 언어 모델)이 긴 텍스트를 처리할 때 발생하는 계산적 한계를 해결하기 위해 제안된 새로운 종류의 시각-언어 모델(VLM)입니다. DeepSeek OCR은 문서 내 텍스트 정보를 이미지(시각 토큰)로 압축하여 적은 수의 토큰으로도 풍부한 정보를 전달할 수 있도록 설계되었습니다.

핵심 구성요소는 DeepEncoder와 DeepSeek3B-MoE-A570M 디코더로, DeepEncoder는 고해상도 입력에도 적은 비전 토큰만 활용할 수 있도록 토큰을 압축하는 역할을 합니다. 실험 결과, 10배 미만의 압축률에서는 97%의 정확도를 제공하고, 20배 압축률에서도 60%의 정확도를 달성할 만큼 성능이 높습니다.

이 모델은 일반적인 문서 텍스트뿐만 아니라 차트, 수식, 도형 등의 다양한 시각 정보도 분석할 수 있으며, 실제 LLM 혹은 VLM의 학습 데이터로 대규모 페이지를 생성하는 실용성도 갖추고 있습니다.


In [55]:
# 두 번째 질문
question2 = "이 논문에서 사용된 벤치마크는 무엇이고, 결과는 어때?"

print(f"💬 질문: {question2}\n")
print("=" * 50)

result2 = await ask_rag_agent(question2)

print(f"\n🤖 답변:\n{result2.final_output}")

💬 질문: 이 논문에서 사용된 벤치마크는 무엇이고, 결과는 어때?


🤖 답변:
이 논문에서 사용된 주요 벤치마크는 Fox 벤치마크(Fox benchmark)와 OmniDocBench입니다.

1. Fox 벤치마크:
- 다양한 문서 레이아웃을 다루는 데이터셋입니다.
- 논문에서는 비전 토큰(visual token) 기반의 압축 성능과 텍스트 복원 정확도를 평가하는 데 사용되었습니다.
- 결과: 비전 토큰이 텍스트 토큰 수의 1/10 수준(압축률 9-10배)일 때 96% 이상의 OCR 디코딩 정확도를 달성하였고, 10-12배 압축에서는 약 90%, 20배 압축에서는 약 60%의 정확도를 기록했습니다.

2. OmniDocBench:
- 다양한 문서 및 OCR 모델들을 비교할 수 있는 벤치마크입니다.
- DeepSeek-OCR은 최신 end-to-end 모델들 중 가장 적은 비전 토큰 수를 사용하면서도 최고 수준의 성능을 보였습니다.
- 예시로, DeepSeek-OCR은 100개의 비전 토큰만을 사용해서도 GOT-OCR2.0(페이지당 256 토큰 사용)을 능가했으며, 800개 미만의 토큰으로 MinerU2.0(평균 6000+ 토큰 사용)도 능가했습니다.

즉, DeepSeek-OCR은 적은 비전 토큰만을 사용하면서도 기존 모델들보다 매우 높은 압축 효율과 우수한 인식 성능을 동시에 달성한 것이 특징입니다.


In [56]:
# 검색 결과 상세 확인
print("📋 실행 결과 상세:")
print("=" * 50)

for idx, item in enumerate(result2.new_items):
    print(f"\n[Item {idx + 1}]")
    print(f"Type: {type(item).__name__}")
    if hasattr(item, 'raw_item'):
        print(f"Raw: {str(item.raw_item)[:500]}...")

📋 실행 결과 상세:

[Item 1]
Type: ToolCallItem
Raw: ResponseFileSearchToolCall(id='fs_05af7fb979ab25df0069625929153481a2be0fd34fb49f3bde', queries=['사용된 벤치마크', '벤치마크 결과', 'benchmarks used', 'benchmark results'], status='completed', type='file_search_call', results=[Result(attributes={}, file_id='file-BEUc9vAdrmPsMZmXSc9pjT', filename='DeepSeek_OCR_paper.pdf', score=0.5328, text='DeepSeek-OCR: Contexts Optical Compression\r\nHaoran Wei, Yaofeng Sun, Yukun Li\r\nDeepSeek-AI\r\nAbstract\r\nWe present DeepSeek-OCR as an initial investigation into the...

[Item 2]
Type: MessageOutputItem
Raw: ResponseOutputMessage(id='msg_05af7fb979ab25df006962592b0e1881a2a013552ce0d2d959', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-BEUc9vAdrmPsMZmXSc9pjT', filename='DeepSeek_OCR_paper.pdf', index=549, type='file_citation')], text='이 논문에서 사용된 주요 벤치마크는 Fox 벤치마크(Fox benchmark)와 OmniDocBench입니다.\n\n1. Fox 벤치마크:\n- 다양한 문서 레이아웃을 다루는 데이터셋입니다.\n- 논문에서는 비전 토큰(visual token) 기반의 압축 성능과 텍스

### 💡 Part A 정리: FileSearchTool 기반 RAG

1. **간단한 설정**: Vector Store 생성 → 파일 업로드 → 에이전트 생성
2. **자동 검색**: LLM이 질문을 분석하여 자동으로 검색 쿼리 생성
3. **호스팅 인프라**: OpenAI가 벡터 저장소와 검색을 관리
4. **쉬운 확장**: 파일 추가만으로 지식베이스 확장 가능

---
# 🔄 Part B: 멀티 에이전트 RAG 시스템

## 6. Handoff 개요

**Handoff**는 에이전트 간에 대화 제어권을 이전하는 메커니즘입니다.

### Handoff 사용 사례:
- **Triage Agent**: 질문 유형에 따라 전문가 에이전트로 라우팅
- **Escalation**: 복잡한 질문은 상위 에이전트로 전달
- **Specialization**: 각 분야별 전문 에이전트 활용

```
┌─────────────────┐
│  Triage Agent   │
│  (라우터 역할)   │
└────────┬────────┘
         │
    ┌────┼────┐
    ▼    ▼    ▼
┌──────┐┌──────┐┌──────┐
│ RAG  ││일반AI ││요약   │
│Agent ││Agent ││Agent │
└──────┘└──────┘└──────┘
```

---
## 7. 전문가 에이전트 정의

각각의 역할을 담당하는 **전문가 에이전트**들을 정의합니다.

In [57]:
# 1. RAG 전문가 에이전트 (문서 기반 답변)
rag_specialist = Agent(
    name="RAG Specialist",
    handoff_description="DeepSeek OCR 논문에 대한 질문을 처리하는 전문가. 논문 내용, 기술적 세부사항, 벤치마크 결과 등에 대한 질문에 답변합니다.",
    instructions="""You are a RAG specialist that answers questions based on the DeepSeek OCR paper.
    
    Your role:
    - Search the vector store for relevant information
    - Provide accurate, document-based answers
    - Include specific details, numbers, and quotes when available
    - Answer in Korean
    """,
    tools=[
        FileSearchTool(
            vector_store_ids=[vector_store.id],
            max_num_results=5,
            include_search_results=True
        )
    ]
)

print("✅ RAG Specialist 에이전트 생성 완료")

✅ RAG Specialist 에이전트 생성 완료


In [58]:
# 2. 일반 AI 에이전트 (일반적인 질문 처리)
general_assistant = Agent(
    name="General Assistant",
    handoff_description="일반적인 질문이나 인사, 잡담 등을 처리하는 어시스턴트. 문서와 관련 없는 일반적인 대화를 담당합니다.",
    instructions="""You are a friendly general assistant.
    
    Your role:
    - Handle greetings and casual conversations
    - Answer general knowledge questions
    - Provide helpful responses for non-document queries
    - Be friendly and conversational
    - Answer in Korean
    """
)

print("✅ General Assistant 에이전트 생성 완료")

✅ General Assistant 에이전트 생성 완료


In [59]:
# 3. 요약 전문가 에이전트
summarizer = Agent(
    name="Summarizer",
    handoff_description="논문의 전체 요약이나 특정 섹션 요약을 요청할 때 사용하는 요약 전문가.",
    instructions="""You are a summarization expert for the DeepSeek OCR paper.
    
    Your role:
    - Search the document and create comprehensive summaries
    - Provide structured summaries with key points
    - Highlight important findings and contributions
    - Use bullet points for clarity
    - Answer in Korean
    """,
    tools=[
        FileSearchTool(
            vector_store_ids=[vector_store.id],
            max_num_results=10,  # 요약을 위해 더 많은 결과
            include_search_results=True
        )
    ]
)

print("✅ Summarizer 에이전트 생성 완료")

✅ Summarizer 에이전트 생성 완료


---
## 8. Triage 에이전트 설정

**Triage Agent**는 사용자 질문을 분석하여 적절한 전문가 에이전트로 **라우팅**합니다.

In [60]:
# Triage 에이전트 생성
triage_agent = Agent(
    name="Triage Agent",
    instructions="""You are a triage agent that routes user questions to the appropriate specialist.
    
    Routing rules:
    1. **RAG Specialist**: Questions about the DeepSeek OCR paper content, technical details, methodology, experiments, or results
    2. **Summarizer**: Requests for summaries, overviews, or key takeaways from the paper
    3. **General Assistant**: Greetings, general questions, or anything not related to the document
    
    Always analyze the user's intent carefully before routing.
    Do NOT answer questions directly - always hand off to the appropriate specialist.
    """,
    handoffs=[rag_specialist, summarizer, general_assistant]
)

print("✅ Triage Agent 생성 완료!")
print(f"   - Handoffs: {[h.name for h in triage_agent.handoffs]}")

✅ Triage Agent 생성 완료!
   - Handoffs: ['RAG Specialist', 'Summarizer', 'General Assistant']


---
## 9. 멀티 에이전트 RAG 실행

이제 **멀티 에이전트 시스템**을 실행하여 다양한 유형의 질문을 테스트합니다.

In [61]:
async def ask_multi_agent(question: str):
    """멀티 에이전트 시스템에 질문하기"""
    with trace("Multi-Agent RAG"):
        result = await Runner.run(triage_agent, question)
        return result

# 테스트 1: 일반 인사 (→ General Assistant)
test1 = "안녕하세요! 오늘 기분이 어때요?"

print(f"💬 질문: {test1}")
print("=" * 50)

result = await ask_multi_agent(test1)
print(f"\n🤖 답변:\n{result.final_output}")
print(f"\n📍 최종 에이전트: {result.last_agent.name}")

💬 질문: 안녕하세요! 오늘 기분이 어때요?

🤖 답변:
안녕하세요! 저는 오늘 기분이 아주 좋아요. 당신은 오늘 어떠세요? 궁금한 것이나 이야기하고 싶은 주제가 있으면 언제든 말씀해 주세요! 😊

📍 최종 에이전트: General Assistant


In [62]:
# 테스트 2: 논문 내용 질문 (→ RAG Specialist)
test2 = "DeepSeek OCR의 핵심 기술은 무엇인가요?"

print(f"💬 질문: {test2}")
print("=" * 50)

result = await ask_multi_agent(test2)
print(f"\n🤖 답변:\n{result.final_output}")
print(f"\n📍 최종 에이전트: {result.last_agent.name}")

💬 질문: DeepSeek OCR의 핵심 기술은 무엇인가요?

🤖 답변:
DeepSeek OCR의 핵심 기술은 다음과 같습니다.

1. **DeepEncoder 아키텍처**: DeepSeek OCR의 핵심 엔진인 DeepEncoder는 고해상도 입력에서도 낮은 활성 메모리와 최소한의 비전 토큰을 유지하는 것이 특징입니다. DeepEncoder는 윈도우 어텐션(window attention) 기반 시각 인식(feature extraction)과 글로벌 어텐션(global attention) 기반 시각 지식 추출 부품으로 구성되어 있습니다. 두 부품 사이에는 16배 컨볼루션 다운샘플러가 연결되어 있어 대량의 비전 토큰을 효과적으로 줄입니다. 예를 들어, 1024×1024 해상도의 이미지를 입력하면, 16×16 패치로 나뉘어지고 컨볼루션 계층을 통해 토큰이 16배 더 압축됩니다.

2. **압축 비율(Compression Ratio) 최적화**: DeepSeek OCR은 비전-텍스트 토큰 압축 비율을 최대 20배까지 달성할 수 있으며, 압축 비율이 10배 미만일 때 97% 이상의 OCR 정확도를 보여줍니다. 20배 압축에서도 약 60%의 정확도를 기록하여 효율적이면서도 실용적인 압축을 실현합니다.

3. **엔드투엔드 VLM 구조**: DeepSeek OCR은 DeepEncoder와 DeepSeek3B-MoE(570M 활성 파라미터의 mixture-of-experts 구조)를 결합한 엔드투엔드 비전-언어 모델입니다. 이 구조를 통해 비전 토큰의 수를 현저히 줄이면서도 오랜 맥락(long context)에 대한 효과적인 문맥 처리가 가능합니다.

4. **실전 활용과 범용성**: DeepSeek OCR은 차트, 화학식, 기하도형, 자연 이미지 등 다양한 요소의 파싱이 가능할 만큼 실용성이 높습니다. 또한, OmniDocBench 등 다양한 벤치마크에서도 적은 비전 토큰(예: 100개만 사용)으로 기존 SOTA 모델을 능가하는 성능을 나타냅니다. 실제 서비스 환경에서도

In [63]:
# 테스트 3: 요약 요청 (→ Summarizer)
test3 = "이 논문 전체를 3줄로 요약해줘"

print(f"💬 질문: {test3}")
print("=" * 50)

result = await ask_multi_agent(test3)
print(f"\n🤖 답변:\n{result.final_output}")
print(f"\n📍 최종 에이전트: {result.last_agent.name}")

💬 질문: 이 논문 전체를 3줄로 요약해줘

🤖 답변:
- DeepSeek-OCR 논문은 시각적(이미지 기반) 정보 압축을 통해 대용량 언어모델(LLM)의 긴 문맥 처리 부담을 혁신적으로 줄일 수 있음을 증명한다.
- DeepSeek-OCR은 DeepEncoder와 DeepSeek3B-MoE 디코더로 구성되며, 10배 정도의 압축에서도 96~97%의 OCR 정확도를 달성하고, 20배 압축에서도 60%의 인식률을 보인다.
- 실제 벤치마크에서 타 모델 대비 훨씬 적은 vision token만으로 SOTA 성능을 증명했으며, 긴 문맥/기억 최적화와 다양한 실용적 문서처리(도표, 수식 등) 가능성을 보여준다.

📍 최종 에이전트: Summarizer


In [64]:
# 테스트 4: 상세 기술 질문 (→ RAG Specialist)
test4 = "OmniDocBench에서 DeepSeek OCR의 성능은 어떤가요? 다른 모델과 비교해서 알려주세요."

print(f"💬 질문: {test4}")
print("=" * 50)

result = await ask_multi_agent(test4)
print(f"\n🤖 답변:\n{result.final_output}")
print(f"\n📍 최종 에이전트: {result.last_agent.name}")

💬 질문: OmniDocBench에서 DeepSeek OCR의 성능은 어떤가요? 다른 모델과 비교해서 알려주세요.

🤖 답변:
DeepSeek OCR는 OmniDocBench 벤치마크에서 강력한 성능을 보입니다. 주요 비교 내용을 정리하면 아래와 같습니다.

- DeepSeek OCR는 100개 vision token만으로 GOT-OCR2.0(256 토큰 사용)을 능가하며, 400개(실효 285개) vision token 사용 시 최첨단(state-of-the-art) 모델과 동등한 성능에 도달합니다. 800개 미만 vision token을 사용하는 Gundam 모드에서는 평균 7,000개 vision token을 사용하는 MinerU2.0도 능가합니다.
- 실제 편집 거리(edit distance) 기준으로, DeepSeek OCR의 다양한 버전은 다음과 같은 전체 성능을 기록합니다(작을수록 성능 우수):
  - Tiny: 0.32
  - Small: 0.205
  - Base: 0.156
  - Large: 0.117
  - Gundam: 0.083
  - Gundam-M: 0.077

- 특히 Gundam-M 모드에서 edit distance가 0.077로, 현존 최고 수준의 정확도를 기록합니다. 이는 MinerU2.0, OLMOCR, Qwen2.5-VL-7B, OCRFlux-3B, InternVL2/3 등 최신 모델 대비 더 적은 vision token으로 유사 혹은 더 우수한 결과를 달성함을 의미합니다.

- 추가적으로, DeepSeek OCR는 일부 문서 유형(예: 슬라이드, 책, 리포트 등)에서는 64~100 vision token만으로도 매우 우수한 성능을 냅니다. 신문과 같은 복잡한 유형은 더 많은 token(Gundam 모드 이상)이 필요합니다.

요약: DeepSeek OCR는 OmniDocBench 벤치마크에서 적은 vision token으로 GOT-OCR2.0, MinerU2.0 등 기존 최고 모델들을 뛰어넘거나 동등한 수준의 높은 OC

---
## 🧹 리소스 정리

실습이 끝나면 생성한 Vector Store를 삭제하여 비용을 절약할 수 있습니다.

In [ ]:
# Vector Store 삭제 (선택사항)
# 주의: 이 셀을 실행하면 Vector Store가 영구 삭제됩니다!

# cleanup = input("Vector Store를 삭제하시겠습니까? (yes/no): ")
# if cleanup.lower() == "yes":
#     client.vector_stores.delete(vector_store.id)
#     print("🗑️ Vector Store 삭제 완료!")
# else:
#     print("⏭️ 삭제를 건너뛰었습니다.")

print(f"📌 현재 Vector Store ID: {vector_store.id}")
print("   (다음 실습에서 재사용하려면 이 ID를 저장해두세요)")

---
# 📊 Part C: 트레이싱 (Tracing)

## 10. 트레이싱 개요

OpenAI Agents SDK에는 **내장 트레이싱 기능**이 포함되어 있습니다.

### 트레이싱이란?
에이전트 실행 중 발생하는 **모든 이벤트를 기록**하는 기능입니다:
- LLM 생성 (Generation)
- 도구 호출 (Tool Calls)
- 핸드오프 (Handoffs)
- 가드레일 (Guardrails)
- 사용자 정의 이벤트

### 트레이싱 구조
```
Trace (워크플로우 전체)
├── Agent Span (에이전트 실행)
│   ├── Generation Span (LLM 호출)
│   ├── Function Span (도구 호출)
│   └── Handoff Span (핸드오프)
└── Agent Span (다음 에이전트)
    └── ...
```

### 주요 속성
| 속성 | 설명 |
|------|------|
| `workflow_name` | 워크플로우 이름 (예: "RAG 챗봇") |
| `trace_id` | 추적 고유 ID |
| `group_id` | 대화 스레드 연결용 ID |
| `metadata` | 사용자 정의 메타데이터 |


---
## 11. 트레이싱 설정 및 실행

### 기본 트레이싱
`trace()` 컨텍스트 매니저를 사용하여 워크플로우를 추적합니다.


In [ ]:
from agents import trace, custom_span
import uuid

# 대화 세션 ID 생성 (여러 트레이스를 연결하기 위함)
session_id = f"session_{uuid.uuid4().hex[:8]}"

print(f"📍 세션 ID: {session_id}")
print("   이 ID로 여러 트레이스를 하나의 대화로 연결할 수 있습니다.")


In [ ]:
async def ask_with_tracing(question: str, workflow_name: str = "RAG Chatbot"):
    """트레이싱이 활성화된 질문 함수"""
    
    # trace() 컨텍스트 매니저로 전체 워크플로우 추적
    with trace(
        workflow_name=workflow_name,
        group_id=session_id,  # 같은 대화 세션 연결
        metadata={
            "question": question,
            "agent_type": "multi-agent"
        }
    ) as current_trace:
        print(f"🔍 Trace ID: {current_trace.trace_id}")
        
        # 사용자 정의 스팬 추가
        with custom_span(name="question_processing"):
            result = await Runner.run(triage_agent, question)
        
        return result, current_trace.trace_id

# 트레이싱 테스트 - 질문 1
print("=" * 60)
print("📊 트레이싱 테스트 1: 논문 질문")
print("=" * 60)

result, trace_id = await ask_with_tracing(
    "DeepSeek OCR의 모델 구조를 설명해줘",
    workflow_name="DeepSeek OCR RAG"
)

print(f"\n🤖 답변:\n{result.final_output}")
print(f"\n📍 최종 에이전트: {result.last_agent.name}")
print(f"🔗 Trace ID: {trace_id}")

In [ ]:
# 트레이싱 테스트 - 질문 2 (같은 세션)
print("=" * 60)
print("📊 트레이싱 테스트 2: 요약 요청 (동일 세션)")
print("=" * 60)

result2, trace_id2 = await ask_with_tracing(
    "이 논문의 주요 기여점을 요약해줘",
    workflow_name="DeepSeek OCR Summary"
)

print(f"\n🤖 답변:\n{result2.final_output}")
print(f"\n📍 최종 에이전트: {result2.last_agent.name}")
print(f"🔗 Trace ID: {trace_id2}")
print(f"\n💡 두 트레이스가 같은 세션({session_id})으로 연결되었습니다!")


In [ ]:
# 여러 에이전트 호출을 하나의 트레이스로 묶기
print("=" * 60)
print("📊 트레이싱 테스트 3: 멀티 스텝 워크플로우")
print("=" * 60)

with trace(
    workflow_name="Multi-Step RAG Workflow",
    group_id=session_id,
    metadata={"step_count": 3}
) as multi_trace:
    
    print(f"🔍 Multi-Step Trace ID: {multi_trace.trace_id}\n")
    
    # Step 1: 일반 질문
    with custom_span(name="step1_greeting"):
        step1 = await Runner.run(triage_agent, "안녕!")
        print(f"Step 1 완료: {step1.last_agent.name}")
    
    # Step 2: 논문 질문
    with custom_span(name="step2_technical"):
        step2 = await Runner.run(triage_agent, "DeepEncoder란 무엇인가요?")
        print(f"Step 2 완료: {step2.last_agent.name}")
    
    # Step 3: 요약 요청
    with custom_span(name="step3_summary"):
        step3 = await Runner.run(triage_agent, "지금까지 대화 내용을 정리해줘")
        print(f"Step 3 완료: {step3.last_agent.name}")

print(f"\n✅ 3개의 스텝이 하나의 트레이스로 기록되었습니다!")
print(f"🔗 Trace ID: {multi_trace.trace_id}")
